In [0]:
dbutils.widgets.text("bronze_tbl", "")
dbutils.widgets.text("bronze_schema", "")
dbutils.widgets.text("silver_tbl", "")
dbutils.widgets.text("silver_schema", "")
dbutils.widgets.text("load_id", "")


In [0]:
bronze_tbl = dbutils.widgets.get("bronze_tbl")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_tbl = dbutils.widgets.get("silver_tbl")
silver_schema = dbutils.widgets.get("silver_schema")
load_id = dbutils.widgets.get("load_id")


In [0]:
df=spark.sql(f"SELECT * FROM {bronze_schema}.{bronze_tbl} WHERE load_id = '{load_id}'")
pre_count = df.count()

In [0]:
#removing duplicate values from the dataframe
df = df.dropDuplicates()

duplicate_records_count = pre_count - df.count()

In [0]:
#schema enforcement
from pyspark.sql.functions import col

df_schema = spark.sql(f"DESCRIBE TABLE {silver_schema}.{silver_tbl}")
for row in df_schema.collect():
    column_name = row['col_name']
    data_type = row['data_type']
    df = df.withColumn(column_name, col(column_name).cast(data_type))


In [0]:
#inserting bronze layer table records into silver layer table records
df.write.mode("overwrite").saveAsTable(f"{silver_schema}.{silver_tbl}")

silver_tbl_records_count = df.count()

In [0]:
dbutils.notebook.exit([duplicate_records_count,silver_tbl_records_count])